## Quarto Experimento: Comparação Triangular por Bairro

In [ ]:
# --- ETAPA 1: PREPARAÇÃO GLOBAL (executada apenas uma vez) ---

print("Preparando dados para a Análise Triangular Focada...")

# Define os padrões de interesse
PADRAO_LUXO = 'P5'
PADRAO_BASICO = 'P1'

# Cria as colunas alvo para as 3 análises
itbi['eh_alto_padrao'] = (itbi['padrao'] == PADRAO_LUXO)
itbi['eh_padrao_basico'] = (itbi['padrao'] == PADRAO_BASICO)
mapa_padrao = {'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5}
itbi['padrao_ordinal'] = itbi['padrao'].map(mapa_padrao)

# Garante que o ano será tratado como categoria
itbi['Ano Avaliacao'] = itbi['Ano Avaliacao'].astype(str)

# Seleciona os bairros de interesse (ex: os 100 maiores)
bairros_para_analise = itbi['Bairro'].value_counts().nlargest(100).index

print(f"Analisando {len(bairros_para_analise)} bairros...")
print("="*80)


# --- ETAPA 2: LOOP PRINCIPAL DE ANÁLISE POR BAIRRO ---

for bairro_foco in bairros_para_analise:
    print(f"\n\n### RELATÓRIO DO BAIRRO: {bairro_foco} ###")
    
    # Filtra o DataFrame para o bairro em questão
    df_bairro = itbi[itbi['Bairro'] == bairro_foco].copy()
    
    if len(df_bairro) < 50:
        print("Dados insuficientes para uma análise robusta. Pulando...")
        print("-" * 80)
        continue

    # --- 2.1 Análise de Padrão MÉDIO (Ordinal/Numérico) ---
    print("\n--- 1. Análise de Padrão MÉDIO ---")
    media_bairro_indice = df_bairro['padrao_ordinal'].mean()
    print(f"Índice de Padrão Médio do Bairro: {media_bairro_indice:.2f}\n")

    include = ['Ano Avaliacao', 'Tipo Construtivo Preponderante', 'idade']
    ignore = list(set(list(itbi)) - set(include))
    task_ordinal = ps.SubgroupDiscoveryTask(
        df_bairro, ps.NumericTarget('padrao_ordinal'),
        ps.create_selectors(df_bairro, ignore=ignore),
        result_set_size=5, depth=2, qf=ps.StandardQFNumeric(a=0.5)
    )
    result_ordinal = ps.BeamSearch().execute(task_ordinal)
    for _, row in result_ordinal.to_dataframe().iterrows():
        print(f"Padrão: {row['subgroup']}")
        print(f"  > Tamanho: {row['size_sg']:,.0f} imóveis | Índice Médio: {row['mean_sg']:.2f}")

    # --- 2.2 Análise de Concentração de ALTO Padrão (P5) ---
    print("\n--- 2. Análise de Concentração de ALTO Padrão (P5) ---")
    total_p5_bairro = df_bairro['eh_alto_padrao'].sum()
    
    if total_p5_bairro > 0:
        cobertura_bairro_p5 = total_p5_bairro / len(df_bairro)
        print(f"Concentração de P5 no Bairro: {cobertura_bairro_p5:.2%} ({total_p5_bairro:,.0f} de {len(df_bairro):,.0f} imóveis)\n")
        
        task_p5 = ps.SubgroupDiscoveryTask(
            df_bairro, ps.BinaryTarget('eh_alto_padrao', True),
            ps.create_selectors(df_bairro, ignore=ignore),
            result_set_size=3, depth=2, qf=ps.WRAccQF()
        )
        result_p5 = ps.BeamSearch().execute(task_p5)
        for _, row in result_p5.to_dataframe().iterrows():
            print(f"Padrão: {row['subgroup']}")
            print(f"  > Concentração P5: {(row['positives_sg']/row['size_sg']):.2%} ({row['positives_sg']:,.0f} de {row['size_sg']:,.0f} imóveis)")
    else:
        print(f"O bairro não possui imóveis de padrão '{PADRAO_LUXO}' para análise.")

    # --- 2.3 Análise de Concentração de Padrão BÁSICO (P1) ---
    print("\n--- 3. Análise de Concentração de Padrão BÁSICO (P1) ---")
    total_p1_bairro = df_bairro['eh_padrao_basico'].sum()

    if total_p1_bairro > 0:
        cobertura_bairro_p1 = total_p1_bairro / len(df_bairro)
        print(f"Concentração de P1 no Bairro: {cobertura_bairro_p1:.2%} ({total_p1_bairro:,.0f} de {len(df_bairro):,.0f} imóveis)\n")
        
        task_p1 = ps.SubgroupDiscoveryTask(
            df_bairro, ps.BinaryTarget('eh_padrao_basico', True),
            ps.create_selectors(df_bairro, ignore=ignore),
            result_set_size=3, depth=2, qf=ps.WRAccQF()
        )
        result_p1 = ps.BeamSearch().execute(task_p1)
        for _, row in result_p1.to_dataframe().iterrows():
            print(f"Padrão: {row['subgroup']}")
            print(f"  > Concentração P1: {(row['positives_sg']/row['size_sg']):.2%} ({row['positives_sg']:,.0f} de {row['size_sg']:,.0f} imóveis)")
    else:
        print(f"O bairro não possui imóveis de padrão '{PADRAO_BASICO}' para análise.")
    
    print("-" * 80)